# OpenSMILE Analysis
This notebook loads OpenSMILE csv- data, cleans and plots it

## Import relevant libraries

In [1]:
import numpy as np
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
import itertools as it
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.api as sm
#import nltk
import scipy.stats as st
import statsmodels.formula.api as smf
import seaborn as sns
import Helper as hp

## Load .csv data with results of OpenSMILE Analysis
First we load .csv data and clean it (removing of NaNs), then we store information of all files in seperate panda dataframes containing information about affect, emotion and valence/arousal for all participants.

In [2]:
data = pd.read_csv("UIST2019_OpenSMILE.csv")

#Set Labels 
emotion_label = ['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness']
affect_label = ['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired']
loi_label = ['Disinterest', 'Normal', 'High Interest']

#Get specific data and save it into new data frames
# We use the pandas .copy(deep=True) function to prevent the SettingWithCopyWarning we would otherwise get. Since we do
# not write, but only read from the data, the warning does not affect the data frames
df_emotion = data[['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness', 'Filename']].copy(deep=True)
df_affect = data[['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired', 'Filename']].copy(deep=True)
df_loi = data[['Disinterest', 'Normal', 'High Interest', 'Filename']].copy(deep=True)
df_ar_val = data[['Arousal', 'Valence', 'Filename']].copy(deep=True)
#For further usage, we want to append the CharacterID as a column, which is saved with other information in the filename
#Since we only want the digits, we can remove all non-digit characters of the filename column and append the column to the df

df_emotion['Char_ID'] = df_emotion['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_affect['Char_ID'] = df_affect['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_loi['Char_ID'] = df_loi['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_ar_val['Char_ID'] = df_ar_val['Filename'].replace('\D+','', regex = True).copy(deep=True)

## Let's load information about the speakers
The speaker ID is saved in a single .csv file containing four important columns: ID, Age, Sex and Acadedmic Status. Since before loaded OpenSMILE csv files are named using the corresponding index (ex. speaker with id 0 has two files 0_a.csv and 0_b.csv), so that a link can be created

In [3]:
char_data = pd.read_csv("UIST2019_CharacterData.csv")  

#Join above tables and Character Tables

#To Join DataFrames we have to cast the column on which we want to join to int, so that both columns have the same data type
char_data['ID'] = char_data['ID'].astype(int)
df_ar_val['Char_ID'] = df_ar_val['Char_ID'].astype(int)
df_emotion['Char_ID'] = df_emotion['Char_ID'].astype(int)
df_affect['Char_ID'] = df_affect['Char_ID'].astype(int)
df_loi['Char_ID'] = df_loi['Char_ID'].astype(int)

#Safe new data frames
df_ar_val_char = df_ar_val.merge(char_data, how = 'left', left_on='Char_ID', right_on='ID')
df_emotion_char = df_emotion.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_affect_char = df_affect.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_loi_char = df_loi.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')

## Chi-squared Test of Independence
We Start with characteristic sex. The null hypothesis states that the two categorical variables sex and e.g. emotion are independent.

We bin the data for each specific voice feature e.g. the emotion anger into quartiles (<= 0.25; <= 0.50 && > 0.25; <=0.75 && > 0.5; <= 1.0 && > 0.75) and use the resulting tables as frequency tables for chi2.

CAREFUL! The below printed results cannot be used for evaluation, since there are frequency counts of zero resulting in an error! This is why there is this line of code 'tables += 5' in the hp.chi2 function, to prevent that error!

In [4]:
print('EMOTION\n')
emo_sex_chi2 = hp.chi2(df_emotion_char, emotion_label,'Sex',  True)
print('\nAFFECT\n')
aff_sec_chi2 = hp.chi2(df_affect_char, affect_label,'Sex',  True)
print('\nAROUSAL-VALENCE\n')
ar_val_sec_chi2 = hp.chi2(df_ar_val_char, ['Arousal', 'Valence'], 'Sex', True)
print('\nLEVEL OF INTEREST\n')
loi_sec_chi2 = hp.chi2(df_loi_char, loi_label, 'Sex', True)

EMOTION

Chi square of Anger : 13.896067883090234 with p-value of: 0.0030500903047402522
Chi square of Boredom : 12.833311837261014 with p-value of: 0.005011301518745974
Chi square of Disgust : 47.640230828188734 with p-value of: 2.539962703251013e-10
Chi square of Fear : 13.896067883090234 with p-value of: 0.0030500903047402522
Chi square of Happiness : 13.896067883090234 with p-value of: 0.0030500903047402522
Chi square of Emo_Neutral : 13.896067883090234 with p-value of: 0.0030500903047402522
Chi square of Sadness : 46.97515917482741 with p-value of: 3.518151711795612e-10

AFFECT

Chi square of Aggressiv : 18.849512259750185 with p-value of: 0.0002937018347639204
Chi square of Cheerful : 4.367669380896267 with p-value of: 0.224401974423878
Chi square of Intoxicated : 8.281611865799036 with p-value of: 0.04053642563437645
Chi square of Nervous : 9.551577667714549 with p-value of: 0.02278894397981628
Chi square of Aff_Neutral : 9.128480408857016 with p-value of: 0.02763055347355608
Ch

Now move on to academic status, the hypothesis being that the variables academic status and e.g. emotion are independent.

In [5]:
print('EMOTION\n')
emo_aca_chi2 = hp.chi2(df_emotion_char, emotion_label,'Academic' , True)
print('\nAFFECT\n')
aff_aca_chi2 = hp.chi2(df_affect_char, affect_label,'Academic', True)
print('\nAROUSAL-VALENCE\n')
ar_val_aca_chi2 = hp.chi2(df_ar_val_char, ['Arousal', 'Valence'],  'Academic',True)
print('\nLEVEL OF INTEREST\n')
loi_aca_chi2 = hp.chi2(df_loi_char, loi_label,'Academic', True)

EMOTION

Chi square of Anger : 2.3895790376252983 with p-value of: 0.49557742956461914
Chi square of Boredom : 2.3895790376252983 with p-value of: 0.49557742956461914
Chi square of Disgust : 1.1324209419594558 with p-value of: 0.7692538606681247
Chi square of Fear : 2.3895790376252983 with p-value of: 0.49557742956461914
Chi square of Happiness : 2.3895790376252983 with p-value of: 0.49557742956461914
Chi square of Emo_Neutral : 2.3895790376252983 with p-value of: 0.49557742956461914
Chi square of Sadness : 1.0535587324439981 with p-value of: 0.7882953285283588

AFFECT

Chi square of Aggressiv : 1.6786548049911896 with p-value of: 0.641689701192599
Chi square of Cheerful : 4.950049376998965 with p-value of: 0.17549152803676873
Chi square of Intoxicated : 1.16160786354863 with p-value of: 0.7622262776529508
Chi square of Nervous : 1.5189383667572 with p-value of: 0.6779063136121222
Chi square of Aff_Neutral : 2.6130436003085364 with p-value of: 0.4552074226724815
Chi square of Tired : 4

Now let's look if age and e.g. emotion/ affect/ arousal-valence/ level of interest are independent

In [6]:
print('EMOTION\n')
emo_age_chi2 = hp.chi2(df_emotion_char, emotion_label,'Age', True)
print('\nAFFECT\n')
aff_age_chi2 = hp.chi2(df_affect_char, affect_label, 'Age', True)
print('\nAROUSAL-VALENCE\n')
ar_val_age_chi2 = hp.chi2(df_ar_val_char, ['Arousal', 'Valence'],'Age' ,True)
print('\nLEVEL OF INTEREST\n')
loi_age_chi2 = hp.chi2(df_loi_char, loi_label, 'Age',  True)

EMOTION

Chi square of Anger : 36.17527668875551 with p-value of: 2.5486092407738426e-06
Chi square of Boredom : 36.17527668875551 with p-value of: 2.5486092407738426e-06
Chi square of Disgust : 14.540649217616345 with p-value of: 0.02414664425981703
Chi square of Fear : 36.17527668875551 with p-value of: 2.5486092407738426e-06
Chi square of Happiness : 36.17527668875551 with p-value of: 2.5486092407738426e-06
Chi square of Emo_Neutral : 36.17527668875551 with p-value of: 2.5486092407738426e-06
Chi square of Sadness : 14.214015072772998 with p-value of: 0.027334751564579592

AFFECT

Chi square of Aggressiv : 32.90102235384214 with p-value of: 1.0955795189958452e-05
Chi square of Cheerful : 18.395304715646684 with p-value of: 0.005316637100267745
Chi square of Intoxicated : 1.9544824040539268 with p-value of: 0.9238368536110602
Chi square of Nervous : 30.45485650363699 with p-value of: 3.2205307975096964e-05
Chi square of Aff_Neutral : 25.443658826647404 with p-value of: 0.0002825362360

## Post-Hoc tests for age, as it has three different groups

If a significant p-value for the category 'Age' is found, we do not yet know which groups differ significantly from each other, so post-hoc testing is done for this character feature.

CAREFUL! The below printed results cannot be used for evaluation, since there are frequency counts of zero resulting in an error! This is why there is this line of code 'tables += 5' in the hp.chi2 function, to prevent that error!

In [10]:
print('EMOTION\n')
print('post-hoc emotions and different groups')
emo_reject_list, emo_corrected_p_vals, emo_combinations, emo_residuals= hp.chi2_post_hoc(df_emotion_char,emotion_label, 'Age', 'bonferroni', True, True)
print('\nAFFECT\n')
print('\n post-hoc affect and different groups')
aff_reject_list, emo_corrected_p_vals, emo_combinations, aff_residuals = hp.chi2_post_hoc(df_affect_char, affect_label, 'Age' ,'bonferroni', True, True)
print('\nAROUSAL-VALENCE\n')
print('\n post-hoc arousal-valence and different groups')
ar_val_reject_list, ar_val_corrected_p_vals, ar_val_combinations, ar_val_residuals = hp.chi2_post_hoc(df_ar_val_char, ['Arousal', 'Valence'], 'Age', 'bonferroni',True, True)
print('\nLEVEL OF INTEREST\n')
print('\n post-hoc level of intereset and different groups')
loi_reject_list, loi_corrected_p_vals, loi_combinations, loi_residuals = hp.chi2_post_hoc(df_loi_char, loi_label, 'Age', 'bonferroni', True, True)

EMOTION

post-hoc emotions and different groups
Anger
Combinations: [('Young', 'Intermediate'), ('Young', 'Old'), ('Intermediate', 'Old')]
Reject List: [ True  True False]
Corrected p-values: [2.30790022e-03 8.91901006e-07 1.00000000e+00]
Boredom
Combinations: [('Young', 'Intermediate'), ('Young', 'Old'), ('Intermediate', 'Old')]
Reject List: [ True  True False]
Corrected p-values: [2.30790022e-03 8.91901006e-07 1.00000000e+00]
Disgust
Combinations: [('Young', 'Intermediate'), ('Young', 'Old'), ('Intermediate', 'Old')]
Reject List: [False  True False]
Corrected p-values: [0.57965828 0.00981785 1.        ]
Fear
Combinations: [('Young', 'Intermediate'), ('Young', 'Old'), ('Intermediate', 'Old')]
Reject List: [ True  True False]
Corrected p-values: [2.30790022e-03 8.91901006e-07 1.00000000e+00]
Happiness
Combinations: [('Young', 'Intermediate'), ('Young', 'Old'), ('Intermediate', 'Old')]
Reject List: [ True  True False]
Corrected p-values: [2.30790022e-03 8.91901006e-07 1.00000000e+00]
Em